In [15]:
import geopandas as gpd
import os
import pandas as pd
from shapely.geometry import shape

In [ ]:
def Col(df : pd.Dataframe) -> pd.Dataframe:

    return df

### PA utils

In [157]:
def create_lat_long(df):
    """
    Replace line by 4 float (Lat1, Long1, Lat2, Long2) into the whole dataframe
    """

    def get_coords(line):
        """
        Transform a line type into 4 float (Lat1, Long1, Lat2, Long2)
        """
        # On crée un objet géométrique à partir de la linestring
        line = shape(line)
        # On récupère les coordonnées du premier point de la linestring
        longitude_dep, latitude_dep = line.coords[0]
        longitude_fin, latitude_fin = line.coords[1]
        return latitude_dep, longitude_dep, latitude_fin, longitude_fin

    df["coords"] = df["geometry"].apply(get_coords)
    df[["latitude_dep", "longitude_dep", "latitude_fin", "longitude_fin"]] = pd.DataFrame(df["coords"].tolist(), index=df.index)
    df = df.drop("coords", axis=1)
    return df

def preprocessing(df):
    """
    Drop useless columns (too much nans) and drop some nan lines.
    """
    #hesite to drop : regime_d, code_com_d, regime_g, code_come_g
    df = create_lat_long(df) #Transform line to point

    def fill_missing_values(row, f):
        if row[f + "_d"] == None or row[f + "_d"] == "AUCUN":
            row[f + "_d"] = row[f + '_g']
        return row

    df = df.apply(fill_missing_values, axis=1, args=("code_com",))
    df = df.apply(fill_missing_values, axis=1, args=("ame",))
    df = df.apply(fill_missing_values, axis=1, args=("regime",))

    df = df[["code_com_d", "ame_d", "regime_d","date_maj", "latitude_dep", "longitude_dep", "latitude_fin",	"longitude_fin"]]
    df.columns = ["code_com", "ame", "regime","date_maj", "latitude_dep", "longitude_dep", "latitude_fin",	"longitude_fin"]
    df = df.dropna(subset=["code_com"])
    return df

### sqfsdf

In [154]:
df = gpd.read_file(os.path.join("..", "data", "france-20230101-formatted.geojson"))

In [167]:
df[df["ame_d"] != df["ame_g"]][['ame_g', 'ame_d']]

,ame_g,ame_d
14,AUCUN,BANDE CYCLABLE
16,AUCUN,PISTE CYCLABLE
17,DOUBLE SENS CYCLABLE NON MATERIALISE,AUCUN
20,AUCUN,PISTE CYCLABLE
22,AUCUN,PISTE CYCLABLE
...,...,...
273232,AUCUN,BANDE CYCLABLE
273234,AUCUN,BANDE CYCLABLE
273236,AUCUN,BANDE CYCLABLE
273237,BANDE CYCLABLE,AUCUN


In [181]:
import numpy as np 

mask = np.logical_and(np.logical_and(df["ame_d"] != "AUCUN", df["ame_g"] != "AUCUN"), np.logical_and(df["ame_d"] != "AUTRE", df["ame_g"] != "AUTRE"))
mask = np.logical_and(mask, df["ame_d"] != df["ame_g"])

In [182]:
df[mask][['ame_d', 'ame_g']]

,ame_d,ame_g
457,COULOIR BUS+VELO,PISTE CYCLABLE
517,BANDE CYCLABLE,DOUBLE SENS CYCLABLE BANDE
873,BANDE CYCLABLE,PISTE CYCLABLE
1019,BANDE CYCLABLE,PISTE CYCLABLE
1027,BANDE CYCLABLE,PISTE CYCLABLE
...,...,...
272397,BANDE CYCLABLE,DOUBLE SENS CYCLABLE BANDE
272739,BANDE CYCLABLE,DOUBLE SENS CYCLABLE NON MATERIALISE
272740,BANDE CYCLABLE,DOUBLE SENS CYCLABLE NON MATERIALISE
273178,BANDE CYCLABLE,COULOIR BUS+VELO


In [174]:
df.shape[0]

273258

In [158]:
df2 = preprocessing(df)

In [159]:
df2.head(1)

,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin
0,86274,AUTRE,EN AGGLOMERATION,2020-07-28,47.086212,0.005851,47.086436,0.00575


In [ ]:
df2[""]

In [76]:
df2["dep"] = list(map(lambda a : a[:2], df2["code_com_d"]))

In [144]:
for f in ["code_com", "ame", "regime"]:
    for i in range(tdf.shape[0]):
        if tdf.loc[i, f + "_d"] == None:
            tdf.loc[i, f + "_d"] = tdf.loc[i, f + '_g']

In [146]:
df2[df2["dep"] == str(75)]['ame_d'].value_counts()

AUCUN                                            4546
PISTE CYCLABLE                                   2409
COULOIR BUS+VELO                                 1226
BANDE CYCLABLE                                    665
AUTRE                                             513
VOIE VERTE                                        157
AMENAGEMENT MIXTE PIETON VELO HORS VOIE VERTE      85
GOULOTTE                                           50
VELO RUE                                           48
Name: ame_d, dtype: int64

In [151]:
tdf = df2.copy()

KeyError: 'ame_g'

In [150]:
tdf.head(3)

,id_local,id_osm,num_iti,code_com_d,ame_d,regime_d,sens_d,largeur_d,local_d,statut_d,...,d_service,source,project_c,ref_geo,geometry,coords,latitude_dep,longitude_dep,latitude_fin,longitude_fin
0,geovelo_429754890_86274,429754890,None,86274,AUTRE,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.00585 47.08621, 0.00575 47.08644...","(47.0862118, 0.005850700000012, 47.08643619999...",47.086212,0.005851,47.086436,0.005750
1,geovelo_618321946_47165,618321946,3:V80,47165,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.02402 44.52984, 0.02409 44.52989)","(44.52983869999997, 0.0240232, 44.5298946, 0.0...",44.529839,0.024023,44.529895,0.024091
2,geovelo_320086926_65440,320086926,None,65440,VOIE VERTE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.04709 43.22005, 0.04696 43.22039)","(43.220046, 0.047085500000012, 43.2203946, 0.0...",43.220046,0.047086,43.220395,0.046962
